My goal of this project was to predict when I would receive my EAD card from USCIS, but I received it sooner than I thought so the data collecting was not complete to have a solid project. However, the script I wrote could be useful to many people to get a better idea when their case will be processed. I will continue to work on the visulization and forecast in the future when my green card application is sent out. 

In [ ]:
#I referd to https://github.com/co89757/USCISCasePoll/blob/master/poll_uscis.py for the data scraping part.

In [1]:
! pip install pyquery

In [2]:
from pyquery import PyQuery as pq
import requests
import smtplib
import os
import sys
import os.path
import re
import pandas as pd

In [3]:
from optparse import OptionParser
from datetime import datetime, date

In [4]:
def poll_status(casenumber):
    """
    poll USCIS case status given receipt number (casenumber)
    Args:
        param1: casenumber the case receipt number
    Returns:
        a tuple (status, details) containing status and detailed info
    Raise:
        error:
    """
    headers = {
        'Accept': 'text/html, application/xhtml+xml, image/jxr, */*',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language':
        'en-US, en; q=0.8, zh-Hans-CN; q=0.5, zh-Hans; q=0.3',
        'Cache-Control': 'no-cache',
        'Connection': 'Keep-Alive',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Host': 'egov.uscis.gov',
        'Referer': 'https://egov.uscis.gov/casestatus/mycasestatus.do',
        'User-Agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2486.0 Safari/537.36 Edge/13.10586'
    }
    url = "https://egov.uscis.gov/casestatus/mycasestatus.do"
    data = {"appReceiptNum": casenumber, 'caseStatusSearchBtn': 'CHECK+STATUS'}
    res = requests.post(url, data=data, headers=headers)
    doc = pq(res.text)
    status = doc('h1').text()
    code = STATUS_OK if status else STATUS_ERROR
    details = doc('.text-center p').text()
    return (code, status, details)

In [ ]:
# Data transformation and cleaning

In [5]:
def get_action_date(status_detail):
    date_regex = re.compile(r'^On (\w+ +\d+, \d{4})')
    m = date_regex.match(status_detail)
    try:
        datestr = m.group(1)
    except:
        datestr = None
    return datestr

In [6]:
def get_form(status_detail):
    form = re.findall(r'\w+-\d+', status_detail)
    return form

In [7]:
def get_days_since_received(status_detail):
        "parse case status and computes number of days elapsed since case-received"
        date_regex = re.compile(r'^On (\w+ +\d+, \d{4}), .*')
        m = date_regex.match(status_detail)
        try:
            datestr = m.group(1)
        except:
            datestr = None
        if not datestr:
            return -1
        recv_date = datetime.strptime(datestr, "%B %d, %Y").date()
        today = date.today()
        span = (today - recv_date).days
        return span

In [8]:
STATUS_OK = 0
STATUS_ERROR = -1
FILENAME_LASTSTATUS = os.path.join(sys.path[0], "LAST_STATUS_{0}.txt")
mynum = 1990053735 # Change this number accordingly 

In [9]:
# Get n case status
n = 30  # Update n as needed for the number of cases you want to check
## Note that if you continuously scrape the data or set n to a very big number, USCIS will block your IP. 
## Need to use vpn if you collect a large amount of data everyday.
case_nums = ['EAC' + str(i) for i in range(mynum, mynum + 30)] # Need to update 'EAC' to according to your case                                                       
vals = [poll_status(case) for case in case_nums]
df = pd.DataFrame.from_records(vals)
df['case'] = case_nums
# Clean dataframe
df['ActionDate'] = df[2].apply(get_action_date)
df['Form'] = df[2].apply(get_form).apply(''.join)
df['Span'] = df[2].apply(get_days_since_received)
df.columns.values[1] = "Action"
del df[0]
del df[2]  #"Detail status"
df = df.set_index('case')
df['CheckDate'] = date.today()

In [127]:
# Delete rows where no case status detail is available
#df_new = df[df['Span'] != -1]

In [10]:
df

,Action,ActionDate,Form,Span,CheckDate
case,,,,,
EAC1990053735,Request for Additional Evidence Was Sent,"May 14, 2019",I-539,57,2019-07-10
EAC1990053736,Card Was Mailed To Me,"June 4, 2019",I-765,36,2019-07-10
EAC1990053737,Card Was Mailed To Me,"June 3, 2019",I-765,37,2019-07-10
EAC1990053738,Card Was Mailed To Me,"June 4, 2019",I-765,36,2019-07-10
EAC1990053739,Card Was Delivered To Me By The Post Office,"May 1, 2019",,70,2019-07-10
EAC1990053740,Case Rejected Because The Version Of The Form ...,"January 31, 2019",I-102,160,2019-07-10
EAC1990053741,Case Was Received,"January 28, 2019",I-539,163,2019-07-10
EAC1990053742,Case Was Received,"January 28, 2019",I-539,163,2019-07-10
EAC1990053743,Case Was Received,"January 28, 2019",I-539,163,2019-07-10


In [11]:
# Save the dataframe to a csv file if needed
#df.to_csv('USCIS_Status.csv')

In [12]:
# Append new df to existing csv file
#with open('USCIS_Status.csv', 'a') as f:
#    df.to_csv(f, header=False)

In [13]:
# Keep only the form number you are intersted in 
df_765 = df.loc[df['Form'] == 'I-765']  # Can update form name 'I-765' as needed

In [15]:
df_765

,Action,ActionDate,Form,Span,CheckDate
case,,,,,
EAC1990053736,Card Was Mailed To Me,"June 4, 2019",I-765,36,2019-07-10
EAC1990053737,Card Was Mailed To Me,"June 3, 2019",I-765,37,2019-07-10
EAC1990053738,Card Was Mailed To Me,"June 4, 2019",I-765,36,2019-07-10
EAC1990053750,Case Was Rejected Because It Was Improperly Filed,"January 31, 2019",I-765,160,2019-07-10
EAC1990053751,Card Was Mailed To Me,"June 3, 2019",I-765,37,2019-07-10
EAC1990053752,Card Was Mailed To Me,"June 4, 2019",I-765,36,2019-07-10
EAC1990053753,Card Was Mailed To Me,"June 3, 2019",I-765,37,2019-07-10
EAC1990053754,Card Was Mailed To Me,"May 29, 2019",I-765,42,2019-07-10


In [16]:
#df_765_received = df_765.loc[df_765['Action']=='Case Was Received']

In [311]:
#df_new_765_approved = df_new_765.loc[df_new_765['Action'] == 'Case Was Approved']

To be continued...